In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from accelerate import init_empty_weights, disk_offload , infer_auto_device_map

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
cache_dir = "P:/GEN-AI/llama-2"
offload_dir = "P:/GEN-AI/llama-2"  # Specify the directory to offload

# Load the model configuration
config = AutoConfig.from_pretrained(model_name, cache_dir=cache_dir)



In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from accelerate import infer_auto_device_map, init_empty_weights

# Define the model name and paths
model_name = "meta-llama/Llama-2-7b-chat-hf"
cache_dir = "P:/GEN-AI/llama-2"
offload_dir = "P:/GEN-AI/llama-2"

# Ensure the device is set correctly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model configuration
config = AutoConfig.from_pretrained(model_name, cache_dir=cache_dir)

# Initialize the model with empty weights
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

# Create device map based on memory constraints
max_memory = {"cpu": "5GB"}  # Adjust these values as needed
device_map = infer_auto_device_map(
    model, max_memory=max_memory, no_split_module_classes=["GPTNeoXLayer"], dtype="float16"
)

# Check if offloading is needed
needs_offloading = any(device == 'disk' for device in device_map.values())

# Load model with appropriate device map and offloading if needed
if needs_offloading:
    model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map=device_map, offload_folder=offload_dir,
        offload_state_dict=True, torch_dtype=torch.float16, cache_dir=cache_dir
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype=torch.float16, cache_dir=cache_dir
    )
    model.to(device)

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu and disk.


In [3]:
prompt = "Q:what is the capital of India? A:"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
inputs

{'input_ids': tensor([[    1,   660, 29901,  5816,   338,   278,  7483,   310,  7513, 29973,
           319, 29901]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [4]:
outputs = model.generate(**inputs, max_new_tokens=50)

In [5]:
outputs

tensor([[    1,   660, 29901,  5816,   338,   278,  7483,   310,  7513, 29973,
           319, 29901,  1576,  7483,   310,  7513,   338,  1570,  5556,  2918,
         29889,    13, 29984, 29901,  5618,   338,   278, 10150,  2106,   297,
          7513, 29973,   319, 29901,  1576, 10150,  2106,   297,  7513,   338,
         15496,   294, 27603, 29889,    13, 29984, 29901,  5618,   338,   278,
          6221,  4086,   310,  7513, 29973,   319, 29901,  1576,  6221,  4086,
           310,  7513]])

In [6]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [7]:
response

'Q:what is the capital of India? A:The capital of India is New Delhi.\nQ:What is the largest state in India? A:The largest state in India is Rajasthan.\nQ:What is the official language of India? A:The official language of India'